### (1) TF 기반

In [61]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split)                # (1) vectorizer 생성: split 기준을 문자열로 지정 
vectorizer = vectorizer.fit(documents)                           # (2) fit            : 문서와 단어 구조를 동일하게 설정       
doc_term_mat = vectorizer.transform(documents)                   # (3) transform      :문서집합에 대해 (1),(2) 적용

doc_term_mat.toarray()   

array([[1, 2, 0, 1, 0, 3, 2],
       [0, 1, 2, 2, 1, 0, 0],
       [2, 1, 0, 0, 2, 1, 0],
       [0, 2, 0, 1, 0, 1, 1]], dtype=int64)

* `doc_term_mat.toarray()` 코드는 term-document matrix의 구조를 추출하게 한다.
>가로가 term(단어의 순서가 위의 key의 value에 해당), 세로가 document이다. 행렬 내 값은 빈도

* 아래 코드는 위에서 생성한 term-document matrix의 단어 집합을 추출하되 사전형으로 반환한다. 이때 key는 단어(term), value는 단어의 순서(빈도가 아님)이다. 

In [28]:
vocabulary = vectorizer.vocabulary_                              # 단어 집합 생성 
vocabulary 

{'빨강': 3, '하양': 6, '검정': 0, '노랑': 1, '파랑': 5, '보라': 2, '연두': 4}

* 아래 코드는 위에서 생성한 term-document matrix에 등장할 단어를 추출하되 리스트로 반환한다. 따라서 vocabulary의 key가 리스트의 원소에 들어간다. 

In [21]:
words = vectorizer.get_feature_names()

words

['검정', '노랑', '보라', '빨강', '연두', '파랑', '하양']

* 위 코드의 (2) `fit()` 메소드와 (3) `transform()` 메소드를 합친 것이 아래의 `fit_transform()` 메소드이다. 따라서 위와 동일한 결과가 생성된다.

In [57]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split)
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()                              # sparse matrix 형태로 반환

array([[1, 2, 0, 1, 0, 3, 2],
       [0, 1, 2, 2, 1, 0, 0],
       [2, 1, 0, 0, 2, 1, 0],
       [0, 2, 0, 1, 0, 1, 1]], dtype=int64)

문서-단어 행렬은 0 값이 많은 셀을 차지하고 있는 `희소 행렬(sparse matrix)`이다. 희소 행렬은 저장과 연산에 있어서 기억 공간의 낭비가 많기 때문에 이를 여러 가지 방법으로 압축하여 `밀집 행렬(dense matrix)`로 변환하여 사용하는 것이 바람직하다. 

sklearn의 CountVectorizer 클래스 내 fit_transform()함수를 적용하면 밀집 행렬 형태로 반환한다.
* term-document matrix에 대해 `toarray()` 메소드 없이 출력된 행결과 희소 행렬 형태로 출력된 행렬 결과를 비교해보자. 
>앞의 tuple은 (i번째 document, j번째 단어)를 의미하고, 그 다음은 int값은 빈도 수를 의미


In [79]:
print(doc_term_mat)                                 # dense matrix 형태로 반환

  (0, 5)	3
  (0, 1)	2
  (0, 0)	1
  (0, 6)	2
  (0, 3)	1
  (1, 4)	1
  (1, 2)	2
  (1, 1)	1
  (1, 3)	2
  (2, 4)	2
  (2, 5)	1
  (2, 1)	1
  (2, 0)	2
  (3, 5)	1
  (3, 1)	2
  (3, 6)	1
  (3, 3)	1


* 한편 이러한 형태의 구조를 가진 행렬은 그 자체로  의미 파악이 어렵기 때문에  for문과 함께 `indices`, `data`, `indptr` 메소드를 이용하면 유용하다.

In [124]:
for i in doc_term_mat:
    print(i.indices)

[5 1 0 6 3]
[4 2 1 3]
[4 5 1 0]
[5 1 6 3]


In [125]:
for i in doc_term_mat:
    print(i.data)

[3 2 1 2 1]
[1 2 1 2]
[2 1 1 2]
[1 2 1 1]


In [32]:
print(doc_term_mat[0])

  (0, 5)	3
  (0, 1)	2
  (0, 0)	1
  (0, 6)	2
  (0, 3)	1


In [62]:
for i in range(len(doc_term_mat.toarray())):                            # i는 문서 번호
    for j in doc_term_mat[i]:
         for i1, j in zip(j.indices, j.data):
            print("{}\t{}\t{}".format(i, vectorizer.get_feature_names()[i1], j))

0	파랑	3
0	노랑	2
0	검정	1
0	하양	2
0	빨강	1
1	연두	1
1	보라	2
1	노랑	1
1	빨강	2
2	연두	2
2	파랑	1
2	노랑	1
2	검정	2
3	파랑	1
3	노랑	2
3	하양	1
3	빨강	1


* 아래 코드는 `binary=True` 옵션을 지정하여 단어 빈도 여부를 보여준다. (단어가 등장하면 1, 아니면 0) 
> 단어 빈도 정보는 사라지지만 문서의 길이가 짧거나 특정 단어의 출현 여부만 중요할 경우에 이 옵셥을 사용

In [31]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split, binary=True)
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[1, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1]], dtype=int64)

* 아래 코드는 `ngram_range` 옵션을 사용하여 N개의 단어들 하나의 단어처럼 만들어서 기존 단어에 추가하는 코드이다.


In [3]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split, ngram_range=(1,1))                  # uni-gram 설졍
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[1, 2, 0, 1, 0, 3, 2],
       [0, 1, 2, 2, 1, 0, 0],
       [2, 1, 0, 0, 2, 1, 0],
       [0, 2, 0, 1, 0, 1, 1]], dtype=int64)

In [32]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split, ngram_range=(1,2))                  # bi-gram 설졍
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[1, 0, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 3, 0, 0, 2,
        2, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 2, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0,
        0, 0, 0],
       [0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0]], dtype=int64)

In [35]:
vocabulary = vectorizer.vocabulary_                                                   # bi-gram이 단어인 집합 생성 
vocabulary 

{'빨강': 11,
 '하양': 22,
 '검정': 0,
 '노랑': 3,
 '파랑': 18,
 '빨강 하양': 14,
 '하양 하양': 24,
 '하양 검정': 23,
 '검정 노랑': 2,
 '노랑 노랑': 4,
 '노랑 파랑': 7,
 '파랑 파랑': 21,
 '보라': 8,
 '연두': 15,
 '빨강 빨강': 13,
 '빨강 노랑': 12,
 '노랑 보라': 5,
 '보라 보라': 9,
 '보라 연두': 10,
 '연두 연두': 17,
 '연두 노랑': 16,
 '파랑 검정': 19,
 '검정 검정': 1,
 '파랑 노랑': 20,
 '노랑 빨강': 6}

* 한편, 문서-단어 행렬에 포함될 단어를 제한하는 방법에는 크게 3가지가 있다.

>방법 (1): 포함될 단어의 리스트를 CountVectorizer를 초기화할 때에 vocabulary 선택 사항으로 지정

>방법 (2): 제외할 단어, 즉 불용어의 리스트를 stop_words 선택 사항으로 지정

>방법 (3): 문서 집합 전체에서의 단어 빈도를 기준으로 빈도 순위 상위에 있는 단어들만 사용. `max_features = N`.최대  N개의 feature만 선택

In [103]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]

vectorizer = CountVectorizer(tokenizer=str.split, max_features=3)         # (3)번 방법 사용해 단어 빈도가 상위 3으로 제한
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[2, 1, 3],
       [1, 2, 0],
       [1, 0, 1],
       [2, 1, 1]], dtype=int64)

In [37]:
vocabulary = vectorizer.vocabulary_                                     
vocabulary 

{'노랑': 0, '파랑': 1}

In [45]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import CountVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]


my_stop_words = ["파랑"]

vectorizer = CountVectorizer(tokenizer=str.split, stop_words= my_stop_words)         # (2)번 방법 사용해 불용어 지정 
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()              

array([[1, 2, 0, 1, 0, 2],
       [0, 1, 2, 2, 1, 0],
       [2, 1, 0, 0, 2, 0],
       [0, 2, 0, 1, 0, 1]], dtype=int64)

In [46]:
vocabulary = vectorizer.vocabulary_                                     
vocabulary 

{'빨강': 3, '하양': 5, '검정': 0, '노랑': 1, '보라': 2, '연두': 4}

### (2) TF-IDF 기반

* 문서-단어 행렬의 값인 단어 가중치로 `tfdif`를 사용할 경우에는 scikit-learn 라이브러에서 제공하는 `TfdifVectorizer 클래스`를 사용한다.

In [48]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import TfidfVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]


vectorizer = TfidfVectorizer(tokenizer=str.split)
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[0.27413531, 0.36289471, 0.        , 0.22193616, 0.        ,
        0.66580848, 0.54827062],
       [0.        , 0.20431324, 0.78304715, 0.49980885, 0.30868168,
        0.        , 0.        ],
       [0.66322944, 0.21949239, 0.        , 0.        , 0.66322944,
        0.2684707 , 0.        ],
       [0.        , 0.6567178 , 0.        , 0.40163007, 0.        ,
        0.40163007, 0.49609304]])

### (3) 정규화 TF-IDF

* TfidfVectorizer 클래스에서는 정규화(normalization) 기능을 지원한다. 많이 사용하는 정규화는 (1) L1 정규화 (2) L2 정규화이다. 별도 옵션을 지정하지 않으면 default로 L2 정규화 값을 출력한다.

> `L1 정규화`란? 용어 빈도를 해당 문서의 용어 빈도의 절대값의 합계로 나눈 것이다. 단어 빈도의 경우에는 0 이상의 값을 갖기 때문에 절대값이 필요없지만 그렇지 않은 데이터의 처리에도 적용할 수 있도록 L1 정규화에서는 개별 단어 빈도를 더하기 전에 절대값을 취한다. L1 정규화를 식으로 나타내면 다음과 같다.  

$$
    \frac{w}{\sum_{i=1}^{n}|w_i|}
$$


> `L2 정규화`란? L1 정규화와 비슷한데 분모가 유클리드 노름(Euclidean norm)으로 계산한다.유클리드 노름은 벡터의 원소 값들을 제곱하여 합한 뒤 제곱근을 취한 값으로 이를 벡터의 길이, 혹은 크기(magnitude)라고 부른다. L2 정규화를 식으로 보이면 다음과 같다.

$$
    \frac{w}{\sqrt{\sum_{i=1}^{n}w_i^2}}
$$

In [49]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import TfidfVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]


vectorizer = TfidfVectorizer(tokenizer=str.split, norm="l1")
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[0.13223798, 0.17505392, 0.        , 0.10705804, 0.        ,
        0.32117411, 0.26447595],
       [0.        , 0.1137696 , 0.43603126, 0.27831311, 0.17188603,
        0.        , 0.        ],
       [0.36553208, 0.12097097, 0.        , 0.        , 0.36553208,
        0.14796486, 0.        ],
       [0.        , 0.33573311, 0.        , 0.2053249 , 0.        ,
        0.2053249 , 0.25361709]])

In [51]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import TfidfVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]


vectorizer = TfidfVectorizer(tokenizer=str.split, norm="l2")        # norm="l2" 를 지정하지 않을 때와 출력 값 동일
doc_term_mat = vectorizer.fit_transform(documents)

doc_term_mat.toarray()

array([[0.27413531, 0.36289471, 0.        , 0.22193616, 0.        ,
        0.66580848, 0.54827062],
       [0.        , 0.20431324, 0.78304715, 0.49980885, 0.30868168,
        0.        , 0.        ],
       [0.66322944, 0.21949239, 0.        , 0.        , 0.66322944,
        0.2684707 , 0.        ],
       [0.        , 0.6567178 , 0.        , 0.40163007, 0.        ,
        0.40163007, 0.49609304]])

* use_idf 선택 사항을 이용하면 TFIDF 계산에서 IDF를 곱하지 않도록 할 수도 있다. 

>use_idf=False, norm=None 선택 사항을 지정하면 CountVectorizer로 생성한 문서-단어 행렬과 똑같은 문서-단어 행렬을 얻을 수 있다.


In [56]:
"""scikit-learn 라이브러리를 이용한 문서-단어 행렬 생성"""

from sklearn.feature_extraction.text import TfidfVectorizer

documents = ["빨강 하양 하양 검정 노랑 노랑 파랑 파랑 파랑", 
             "빨강 빨강 노랑 보라 보라 연두",
             "연두 연두 노랑 파랑 검정 검정",
              "노랑 파랑 노랑 빨강 하양"]


count_vectorizer = CountVectorizer(tokenizer=str.split)
count_doc_term_mat = vectorizer.fit_transform(documents)
tfidf_vectorizer = TfidfVectorizer(tokenizer=str.split, use_idf=False, norm=None)       # IDF를 곱하지 않고, 정규화도 하지 않음
count_doc_term_mat = count_vectorizer.fit_transform(documents)
tfidf_doc_term_mat = tfidf_vectorizer.fit_transform(documents)

In [57]:
count_doc_term_mat.toarray()

array([[1, 2, 0, 1, 0, 3, 2],
       [0, 1, 2, 2, 1, 0, 0],
       [2, 1, 0, 0, 2, 1, 0],
       [0, 2, 0, 1, 0, 1, 1]], dtype=int64)

In [58]:
tfidf_doc_term_mat.toarray()

array([[1., 2., 0., 1., 0., 3., 2.],
       [0., 1., 2., 2., 1., 0., 0.],
       [2., 1., 0., 0., 2., 1., 0.],
       [0., 2., 0., 1., 0., 1., 1.]])

### 실제 텍스트를 불러와 TF, TF-IDF 기반의 term-document matrix 생성

In [1]:
"""  형태소 분석 후 파일을 불러와 TF기반의 term-document matrix를 생성"""

from operator import itemgetter
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import ujson

FEATURE_POS = ["Noun", "Verb", "Adverb", "Adjective", "Exclamation", "Determiner"]
POS_KEY = "review_pos"

def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들인후 CountVectorizer에 입력할 파일 형태로 document set 집합을 생성"""
    
    documents = []

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            json_obj = ujson.loads(line)
            text_pos = json_obj[POS_KEY]
            
            words = []
            
            for sent_pos in text_pos:
                for word, pos in sent_pos:
                    if pos not in FEATURE_POS:
                        continue

                    words.append(word)

            document = " ".join(words)
            documents.append(document)
            
    documents = np.asarray(documents)
            
    return documents

            
def main():
    """TF 기반의 term-document matrix를 생성하여 TSV 형태로 파일 저장"""
    
    input_file_name = r"pos_movie.txt"
    output_file_name = "tf_pos_movie.txt"
    documents = read_documents(input_file_name)
    
    vectorizer = CountVectorizer(tokenizer=str.split, max_features =100)
    doc_term_mat = vectorizer.fit_transform(documents)   
   
    with open(output_file_name, "w", encoding="utf-8") as output_file:
        doc_num = len(doc_term_mat.toarray())
        for i in range(doc_num):                            # i는 문서 번호
            for j in doc_term_mat[i]:
                for i1, j in zip(j.indices, j.data):
                    print("{}\t{}\t{}".format(i, vectorizer.get_feature_names()[i1], j), file = output_file)
    
main()
